Construction du corpus
==

In [1]:
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import os
import time

In [2]:
link = "https://agriculture.gouv.fr/les-politiques-agricoles-travers-le-monde"

In [3]:
def get_country_profile(link:str) -> list:
    
    import requests
    from bs4 import BeautifulSoup
    startTime = time.time()
    
    site = requests.get(link)
    
    if site.ok:
        drapeau = []
        pays = []
        info = []
        soup = BeautifulSoup(site.text,'lxml')
        tbody = soup.find('tbody')
        for first in tbody.find_all("tr",class_="row_even"):
            for c in first.find_all("figure"):
                try:
                    drap1 = c.find('img')
                    drapeau.append(drap1["src"])
                except Exception as e:
                    print(e)
            for c in first.find_all("a"):
                try:
                    name1 = (c.string if c else '')
                    pays.append(name1)
                    if c["href"].find('http://agriculture.gouv.fr') != -1:
                        lien1 = c["href"]
                    else:
                        lien1 = 'http://agriculture.gouv.fr'+ c["href"]
                    info.append(lien1)
                except Exception as e:
                    print(e)
        for second in tbody.find_all("tr",class_="row_odd"):
            for c in second.find_all("figure"):
                try:
                    drap2 = c.find('img')
                    drapeau.append(drap2["src"])
                except Exception as e:
                    print(e)
                
            for c in second.find_all("a"):
                try:
                    name2 = c.string
                    pays.append(name2.strip())
                    if c["href"].find('http://agriculture.gouv.fr') != -1 or c["href"].find('https://agriculture.gouv.fr')!=-1:
                        lien2 = c["href"]
                    else:
                        lien2 = 'http://agriculture.gouv.fr'+ c["href"]
                    info.append(lien2)
                except Exception as e:
                    print(e)
    executionTime = (time.time() - startTime)
    print('Execution time in seconds: ' + str(executionTime))
    
    return pays,info,drapeau

In [4]:
country,info,symbol = get_country_profile(link)

'NoneType' object has no attribute 'strip'
Execution time in seconds: 1.0737147331237793


In [5]:
startTime = time.time()
tup = []
intrus = []
pdf_link = []
for c in info:
    try:
        obj = requests.get(c)
        if obj.ok:
            sp = BeautifulSoup(obj.text,'lxml')
            try:
                div = sp.find("div",class_="field field-name-body field-type-text-with-summary field-label-hidden")
                a = div.find("a")
                if a["href"].find('https://agriculture.gouv.fr') != -1 or a["href"].find('http://agriculture.gouv.fr') != -1:
                    lien = a["href"]
                else:
                    lien = 'https://agriculture.gouv.fr'+ a["href"]
                pdf_link.append(lien)
            except AttributeError:
                intrus.append(c)
                pass
            except TypeError:
                intrus.append(c)
                pass
    except Exception as e:
        intrus.append(c)
        pass
executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

Execution time in seconds: 49.716878175735474


In [6]:
liste = []
for c in os.listdir():
    if c.endswith(".pdf") and "2019" not in c:
        liste.append(c)
        
class_="block-toggle"
class_="page__title title "

In [7]:
tup = []
for c,k in zip(info,country):
    try:
        obj = requests.get(c)
        if obj.ok:
            sp = BeautifulSoup(obj.text,'lxml')
            div = sp.find_all("div",class_="field-item even")
            if """<div class="block-toggle-container">""" in str(div):
                tup.append((k,c))
            else:
                pass
    except Exception as e:
        print(e)

In [9]:
count = 0
for c in tup:
    count += 1
    print(c)
print(count)

('Algérie', 'http://agriculture.gouv.fr/algerie')
('Angola', 'http://agriculture.gouv.fr/angola')
('Kenya', 'http://agriculture.gouv.fr/kenya')
('Lettonie', 'http://agriculture.gouv.fr/lettonie')
('Bénin', 'http://agriculture.gouv.fr/benin')
('Birmanie', 'http://agriculture.gouv.fr/birmanie')
('Brésil', 'http://agriculture.gouv.fr/bresil')
('Bulgarie', 'http://agriculture.gouv.fr/bulgarie-1')
('Mali', 'http://agriculture.gouv.fr/mali')
('Maroc', 'http://agriculture.gouv.fr/maroc')
('Danemark', 'http://agriculture.gouv.fr/danemark-1')
('Egypte', 'http://agriculture.gouv.fr/egypte')
('Portugal', 'http://agriculture.gouv.fr/portugal-1')
('Royaume-Uni', 'http://agriculture.gouv.fr/royaume-uni-0')
('Grèce', 'http://agriculture.gouv.fr/grece')
('Hongrie', 'http://agriculture.gouv.fr/hongrie-0')
('Tunisie', 'http://agriculture.gouv.fr/tunisie')
('Arabie saoudite', 'http://agriculture.gouv.fr/arabie-saoudite')
('Australie', 'http://agriculture.gouv.fr/australie')
('Autriche', 'http://agricultu

In [145]:
len(country),len(info)

(59, 59)

In [70]:
import pdfplumber
lst = []
with pdfplumber.open(liste[14]) as pdf:
    try:
        for i in range(1,75):
            first_page = pdf.pages[i]
            lst.append(first_page.extract_text())
    except Exception as e:
        pass

In [82]:
with open("test","w") as f:
    for c in lst:
        f.write(c.strip('\u2009'))

UnicodeEncodeError: 'charmap' codec can't encode character '\u2009' in position 51: character maps to <undefined>

In [107]:
x0 = 0    # Distance of left side of character from left side of page.
x1 = 0.5  # Distance of right side of character from left side of page.
y0 = 0  # Distance of bottom of character from bottom of page.
y1 = 1  # Distance of top of character from bottom of page.

all_content = []
with pdfplumber.open(liste[0]) as pdf:
    for i, page in enumerate(pdf.pages):
        width = page.width
        height = page.height

        # Crop pages
        left_bbox = (x0*float(width), y0*float(height), x1*float(width), y1*float(height))
        page_crop = page.crop(bbox=left_bbox)
        left_text = page_crop.extract_text()

        left_bbox = (0.5*float(width), y0*float(height), 1*float(width), y1*float(height))
        page_crop = page.crop(bbox=left_bbox)
        right_text = page_crop.extract_text()
        page_context = '\n'.join([left_text, right_text])
        all_content.append(page_context)
        if i > 1:
            print(page_context)

En 2014, les cultures OGM représentent une superficie de
2,7Mha, ce qui place l'Afrique du Sud au 9erang mondial et
au premier rang des pays africains. Les cultures OGM sont
le maïs, le soja et le coton.
Les principales productions animales sont les volailles (17%
de la production agricole en valeur), les bovins viande, le
lait, les ovins viande et les porcins. La production de viandes
toutes espèces confondues est insuffisante pour satisfaire
une demande croissante en raison du changement de r
égime alimentaire induit par la hausse du niveau de vie et
l’urbanisation (63%).
Un vignoble de 100000 ha permet à l'Afrique du Sud d'être
le 8eproducteur mondial de vin en 2014 (11,8Mhl). Après
une année record en 2013, les exportations ont reculé de
20% en volume en 2014 pour atteindre un volume de
4,2Mhl. La production sud-africaine se distingue par son 
ancienneté, l'expérience de la viticulture datant de plus de
350 ans.
Le secteur agricole est en crise en raison de la sécheresse
qui touche

négociations se sont conclues avec le paraphe de l’Accord
par les négociateurs en juillet 2014, la revue juridique en
2015, pour une signature à l’été 2016. Le texte devra ensuite
être ratifié par l’ensemble des parties.
L’Afrique du Sud, dont l’accord prévoit un traitement diffé-
rencié en raison de son niveau de développement, améliore
son accès au marché européen pour 32 produits agricoles
par rapport aux conditions dont elle bénéficie aujourd’hui
dans le cadre de l’ACDC. Les trois principaux produits pour
lesquels elle bénéficiera d’un meilleur accès sont le vin
(1,1 million hl libres de droit contre 350000 aujourd’hui), le
sucre (150000 t libres de droit – rien jusqu’à présent) et
l’éthanol (80000 t libres de droit – rien jusqu’à présent).
L’accord prévoit également un meilleur accès pour les fruits
transformés, les produits laitiers et les fleurs coupées.
L’UE dans le cadre de ces négociations a obtenu un meilleur
accès pour un certains nombre de produits agricoles (blé,
orge, vi

Madagascar - Océan Indien», les deux parties se sont accordées
sur la création d’un institut franco-sud-africain des sciences
agronomiques, F'SAGRI.
Le F’SAGRI est un centre de formation et de recherche
franco/sud-africain dans le domaine agricole dont l’objectif
est de renforcer les capacités d’universités rurales grâce à
la formation d’étudiants aux niveaux master et doctorat,
l’échange d’étudiants et d’enseignants entre la France et
l’Afrique du Sud et le développement de programmes
conjoints de recherche, en lien avec les entreprises du secteur.
La première phase du projet, centrée sur la thématique de
la transformation agro-alimentaire, a débuté en 2015 sur le
campus de l’Université de Fort Hare dans la province de
l’Eastern Cape avec les premiers cours et l’arrivée d’une 
experte mise à disposition par le MAAF pour assurer la 
coordination du projet, rejointe début 2016 par une experte
technique du MAEDI en charge du volet enseignement et 
recherche.
Par ailleurs, le réseau géogr

In [94]:
import logging
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup

logging.basicConfig(
    format='%(asctime)s %(levelname)s:%(message)s',
    level=logging.INFO)

class Crawler:

    def __init__(self, urls=[]):
        self.visited_urls = []
        self.urls_to_visit = urls

    def download_url(self, url):
        return requests.get(url).text

    def get_linked_urls(self, url, html):
        soup = BeautifulSoup(html, 'html.parser')
        for link in soup.find_all('a'):
            path = link.get('href')
            if path and path.startswith('/'):
                path = urljoin(url, path)
            yield path

    def add_url_to_visit(self, url):
        if url not in self.visited_urls and url not in self.urls_to_visit:
            self.urls_to_visit.append(url)

    def crawl(self, url):
        html = self.download_url(url)
        for url in self.get_linked_urls(url, html):
            self.add_url_to_visit(url)

    def run(self):
        while self.urls_to_visit:
            url = self.urls_to_visit.pop(0)
            logging.info(f'Crawling: {url}')
            try:
                self.crawl(url)
            except Exception:
                logging.exception(f'Failed to crawl: {url}')
            finally:
                self.visited_urls.append(url)

if __name__ == '__main__':
    Crawler(urls=['https://www.imdb.com/']).run()

2021-11-19 01:43:10,753 INFO:Crawling: https://www.imdb.com/
2021-11-19 01:43:12,997 INFO:Crawling: https://www.imdb.com/?ref_=nv_home
2021-11-19 01:43:14,319 INFO:Crawling: https://www.imdb.com/calendar/?ref_=nv_mv_cal
2021-11-19 01:43:15,346 INFO:Crawling: https://www.imdb.com/list/ls016522954/?ref_=nv_tvv_dvd
2021-11-19 01:43:20,672 INFO:Crawling: https://www.imdb.com/chart/top/?ref_=nv_mv_250
2021-11-19 01:43:22,974 INFO:Crawling: https://www.imdb.com/chart/moviemeter/?ref_=nv_mv_mpm
2021-11-19 01:43:24,807 INFO:Crawling: https://www.imdb.com/feature/genre/?ref_=nv_ch_gr
2021-11-19 01:43:25,888 INFO:Crawling: https://www.imdb.com/chart/boxoffice/?ref_=nv_ch_cht
2021-11-19 01:43:26,599 INFO:Crawling: https://www.imdb.com/showtimes/?ref_=nv_mv_sh
2021-11-19 01:43:29,685 INFO:Crawling: https://www.imdb.com/movies-in-theaters/?ref_=nv_mv_inth
2021-11-19 01:43:31,158 INFO:Crawling: https://www.imdb.com/coming-soon/?ref_=nv_mv_cs
2021-11-19 01:43:32,920 INFO:Crawling: https://www.imdb.co

2021-11-19 01:44:30,653 INFO:Crawling: https://www.imdb.com/list/ls063853872/mediaviewer/rm3509559553?ref_=hm_edcft_pks_renew_6_t
2021-11-19 01:44:31,217 INFO:Crawling: https://www.imdb.com/family-entertainment-guide/lgbtq-power-couples/rg3163593472/mediaviewer/rm2110191873?ref_=hm_edcft_wot_lgbtq_7_i
2021-11-19 01:44:31,782 INFO:Crawling: https://www.imdb.com/family-entertainment-guide/lgbtq-power-couples/rg3163593472/mediaviewer/rm2110191873?ref_=hm_edcft_wot_lgbtq_7_t
2021-11-19 01:44:32,491 INFO:Crawling: https://www.imdb.com/festival-central/abff-2021-photos-we-love/rg491100928/mediaviewer/rm301263105/?ref_=hm_edcft_scg_hht_8_i
2021-11-19 01:44:33,156 INFO:Crawling: https://www.imdb.com/festival-central/abff-2021-photos-we-love/rg491100928/mediaviewer/rm301263105/?ref_=hm_edcft_scg_hht_8_t
2021-11-19 01:44:33,738 INFO:Crawling: https://www.imdb.com/awards-central/2021-media-access-awards/rg1178966784/mediaviewer/rm1437788417?ref_=hm_edcft_ft_outfest_9_i
2021-11-19 01:44:34,326 INF

KeyboardInterrupt: 